In [1]:
import os
import math
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import tensorflow as tf
from tensorflow import keras
import py
import mylib
import cv2 as cv
import pytesseract
from tqdm import tqdm
from typing import Optional, List, Dict, Set, Tuple
from scml.nlp import strip_punctuation, to_ascii_str

In [2]:
#os.environ["OMP_THREAD_LIMIT"] = "1"
pd.set_option("use_inf_as_na", True)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)

In [3]:
train = pd.read_csv("input/train.csv", engine="c", low_memory=False)
train["target"] = mylib.target_label(train)
train["image_path"] = "input/train_images/" + train["image"]
posting_ids = train["posting_id"].tolist()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34250 entries, 0 to 34249
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   posting_id   34250 non-null  object
 1   image        34250 non-null  object
 2   image_phash  34250 non-null  object
 3   title        34250 non-null  object
 4   label_group  34250 non-null  int64 
 5   target       34250 non-null  object
 6   image_path   34250 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.8+ MB


In [4]:
class ArcMarginProduct(keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [5]:
model_dir = "models/eb0_arc_20210501_0030"
m0 = keras.models.load_model(f"{model_dir}/trial_0/fold_0/model.h5", custom_objects={"ArcMarginProduct": ArcMarginProduct})
m0 = keras.models.Model(inputs=m0.input[0], outputs=m0.get_layer("embedding_output").output)
m0.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 1280)              4049571   
_________________________________________________________________
layer_normalization_1 (Layer (None, 1280)              2560      
_________________________________________________________________
dense_1 (Dense)              (None, 1280)              1639680   
_________________________________________________________________
embedding_output (LayerNorma (None, 1280)              2560      
Total params: 5,694,371
Trainable params: 1,644,800
Non-trainable params: 4,049,571
_________________________________________________________________


In [6]:
m1 = keras.models.load_model(f"{model_dir}/trial_0/fold_1/model.h5", custom_objects={"ArcMarginProduct": ArcMarginProduct})
m1 = keras.models.Model(inputs=m1.input[0], outputs=m1.get_layer("embedding_output").output)
m1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 1280)              4049571   
_________________________________________________________________
layer_normalization_2 (Layer (None, 1280)              2560      
_________________________________________________________________
dense_2 (Dense)              (None, 1280)              1639680   
_________________________________________________________________
embedding_output (LayerNorma (None, 1280)              2560      
Total params: 5,694,371
Trainable params: 1,644,800
Non-trainable params: 4,049,571
_________________________________________________________________


In [7]:
m2 = keras.models.load_model(f"{model_dir}/trial_0/fold_2/model.h5", custom_objects={"ArcMarginProduct": ArcMarginProduct})
m2 = keras.models.Model(inputs=m2.input[0], outputs=m2.get_layer("embedding_output").output)
m2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 1280)              4049571   
_________________________________________________________________
layer_normalization_3 (Layer (None, 1280)              2560      
_________________________________________________________________
dense_3 (Dense)              (None, 1280)              1639680   
_________________________________________________________________
embedding_output (LayerNorma (None, 1280)              2560      
Total params: 5,694,371
Trainable params: 1,644,800
Non-trainable params: 4,049,571
_________________________________________________________________


In [8]:
idg = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    data_format="channels_last",
    dtype=np.float32
)
data = idg.flow_from_dataframe(
    dataframe=train,
    x_col="image",
    y_col="label_group",
    directory="input/train_images",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1024,
    shuffle=False,
    class_mode="raw",
    interpolation="nearest",
)
y0 = m0.predict(data, verbose=1)
y1 = m1.predict(data, verbose=1)
y2 = m2.predict(data, verbose=1)
assert y0.shape == y1.shape == y2.shape
print(f"y0.shape={y0.shape}")

Found 34250 validated image filenames.
34/34 [==============================] - 3135s 92s/step
y0.shape=(34250, 1280)


In [9]:
res = []
for i in range(len(y0)):
    a = np.vstack((y0[i], y1[i], y2[i]))
    m = np.mean(a, axis=0)
    res.append(m)
em = np.array(res, dtype=np.float32)
assert y0.shape == em.shape
print(f"em.shape={em.shape}")

em.shape=(34250, 1280)


In [38]:
%%time
threshold = 0.01
nn = NearestNeighbors(
    n_neighbors=min(49, len(posting_ids) - 1), metric="euclidean"
)
nn.fit(em)
distances, indices = nn.kneighbors()
res: List[List[str]] = [[] for _ in range(len(indices))]
for i in range(len(indices)):
    for j in range(len(indices[0])):
        if distances[i][j] > threshold:
            break
        res[i].append(posting_ids[indices[i][j]])
train["image_matches"] = res

CPU times: user 1min 53s, sys: 49.9 s, total: 2min 43s
Wall time: 1min


th=.25, f1=.586
th=.30, f1=.586
th=.35, f1=.587
th=.40, f1=.583

In [39]:
%%time
train["phash_matches"] = mylib.phash_matches(train, threshold=0.3)

CPU times: user 31.1 s, sys: 15.8 s, total: 46.9 s
Wall time: 47.8 s


In [40]:
%%time
train["title_p"] = train.apply(mylib.preprocess("title"), axis=1)

CPU times: user 36.1 s, sys: 344 ms, total: 36.4 s
Wall time: 37.1 s


In [41]:
%%time
st_name = "stsb-distilbert-base"
#st_name = "paraphrase-distilroberta-base-v1"
#st_name = "paraphrase-xlm-r-multilingual-v1"
train["title_matches"] = mylib.sbert_matches(
    model_path=f"pretrained/sentence-transformers/{st_name}",
    sentences=train["title_p"].tolist(),
    posting_ids=posting_ids,
    threshold=0.5
)

CPU times: user 46min 58s, sys: 1min 20s, total: 48min 19s
Wall time: 10min 54s


In [42]:
def erode_dilate(img):
    kernel = np.ones((2, 2), np.uint8)
    img = cv.erode(img, kernel, iterations=1)
    img = cv.dilate(img, kernel, iterations=1)
    return img


def image_to_text(img_path, mode: str, timeout: float, neighbours: int=41, psm: int=3) -> Optional[str]:
    config = f"--psm {psm}"
    s1, s2 = None, None
    img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
    #img = cv.resize(img, None, fx=0.5, fy=0.5, interpolation=cv.INTER_AREA)
    img = cv.medianBlur(img, 3)
    if mode == "binary_inverted" or mode == "binary":
        th = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, neighbours, 2)
        th = erode_dilate(th)
        try:
            s1 = pytesseract.image_to_string(th, timeout=timeout, config=config)
        except:
            s1 = None
    if mode == "binary_inverted" or mode == "inverted":
        th = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, neighbours, 2)
        th = erode_dilate(th)
        try:
            s2 = pytesseract.image_to_string(th, timeout=timeout, config=config)
        except:
            s2 = None
    if s1 is None and s2 is None:
        return None
    tokens = []
    if s1 is not None:
        s1 = to_ascii_str(s1)
        s1 = strip_punctuation(s1)
        tokens += s1.split()
    if s2 is not None:
        s2 = to_ascii_str(s2)
        s2 = strip_punctuation(s2)
        tokens += s2.split()
    return " ".join(tokens)

In [43]:
OCR = False
if OCR:
    res = []
    n_timeout = 0
    for t in tqdm(train.itertuples()):
        img_path = getattr(t, "image_path")
        s = image_to_text(img_path, mode="inverted", timeout=0.4, neighbours=41, psm=11)
        if s is None:
            s = ""
            n_timeout += 1
        res.append(s)
    print(f"n_timeout={n_timeout}")

In [44]:
if OCR:
    train["itext"] = res
    train["text"] = train["title"] + " " + train["itext"]
    cols = ["text", "itext", "title"]
    train[cols].head()

In [45]:
%%time
if OCR:
    train["text_p"] = train.apply(mylib.preprocess("text"), axis=1)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.7 µs


In [46]:
if OCR:
    st_name = "stsb-distilbert-base"
    #st_name = "paraphrase-distilroberta-base-v1"
    #st_name = "paraphrase-xlm-r-multilingual-v1"
    train["text_matches"] = mylib.sbert_matches(
        model_path=f"pretrained/sentence-transformers/{st_name}",
        sentences=train["text_p"].tolist(),
        posting_ids=posting_ids,
        threshold=0.5
    )

In [47]:
fs = ["phash_matches", "title_matches", "image_matches"]
if OCR:
    fs.append("text_matches")
train["matches"] = train.apply(mylib.combine_as_list(fs), axis=1)
train["f1"] = train.apply(mylib.metric_per_row("matches"), axis=1)
print(f"Combined score={train.f1.mean():.3f}")

Combined score=0.063


In [48]:
res = [
    {
        "score": 0.088,
        "phash_threshold": 0.3,
        "title_threshold": 0.5,
        "image_threshold": 0.5,
        "text_threshold": None,
        "ocr_threshold": None,
        "ocr_timeout": None,
        "ocr_neighbours": None,
        "ocr_psm": None
    },
    {
        "score": 0.674,
        "phash_threshold": 0.3,
        "title_threshold": 0.5,
        "text_threshold": 0.5,
        "image_threshold": 0.5,
        "ocr_threshold": "inverted",
        "ocr_timeout": 0.4,
        "ocr_neighbours": 41,
        "ocr_psm": 11
    },
    {
        "score": 0.674,
        "phash_threshold": 0.3,
        "title_threshold": 0.5,
        "text_threshold": 0.5,
        "image_threshold": 0.5,
        "ocr_threshold": "binary",
        "ocr_timeout": 0.4,
        "ocr_neighbours": 41,
        "ocr_psm": 11
    },
    {
        "score": 0.674,
        "phash_threshold": 0.3,
        "title_threshold": 0.5,
        "image_threshold": 0.5,
        "text_threshold": None,
        "ocr_threshold": None,
        "ocr_timeout": None,
        "ocr_neighbours": None,
        "ocr_psm": None
    }
]
df = pd.DataFrame.from_records(res)
df.sort_values("score", ascending=False, inplace=True, ignore_index=True)
df.T.head(30)

,0,1,2,3
score,0.674,0.674,0.674,0.088
phash_threshold,0.3,0.3,0.3,0.3
title_threshold,0.5,0.5,0.5,0.5
image_threshold,0.5,0.5,0.5,0.5
text_threshold,0.5,0.5,NaN,NaN
ocr_threshold,inverted,binary,None,None
ocr_timeout,0.4,0.4,NaN,NaN
ocr_neighbours,41.0,41.0,NaN,NaN
ocr_psm,11.0,11.0,NaN,NaN


In [49]:
cols = ["f1", "target", "matches"] + fs
train[cols].head(30)

,f1,target,matches,phash_matches,title_matches,image_matches
0,0.015152,"[train_4206743389, train_984412308, train_121852154, train_762535630, train_3517115705, train_2262735115, train_3755300225, train_368430234, train_2317288464, train_3208392688, train_1458933053, train_4282331229, train_3949355563, train_1086352645, train_1446216827, train_2642455336, train_3838978688, train_98545496, train_3569591480, train_797513730, train_863149549, train_1580073987, train_2081303877, train_3194609880, train_678936454, train_2677777576, train_864736904, train_2184916710, train_1551941123, train_902429172, train_3338833167, train_1079287407, train_1966427691, train_2974564568, train_225670337, train_1905044128, train_1806160391, train_592604757, train_2128389901, train_4028145462, train_2333705544, train_686541160, train_2710828031, train_3340371184, train_1724556465, train_3003421228, train_1166656904, train_150004523, train_2057764463, train_1273401808, train_219281992]","[train_920661713, train_2001760144, train_4095331020, train_1292152697, train_1633843449, train_2812635096, train_1038767026, train_3318627594, train_2464356923, train_798983928, train_1930813487, train_4061750790, train_36508312, train_3023369684, train_621862145, train_3198005853, train_2091196525, train_59612027, train_3369186413, train_3563363070, train_1984944968, train_4074846898, train_1225784982, train_1806152124, train_2330208712, train_3976679470, train_3749487198, train_3346806089, train_3015101622, train_651460400, train_1449806723, train_4016582588, train_3210343892, train_3171305414, train_3261237801, train_400379362, train_2962857384, train_2406599165, train_3855055492, train_4030248161, train_2658426624, train_4248767492, train_1835268339, train_511692366, train_1911257755, train_3205349916, train_475857941, train_379528076, train_4200009730, train_2288590299, train_984412308, train_2304760112, train_59243023, train_194519968, train_3412174951, train_3383937433, train_1512475953, train_4176745626, train_830884721, train_586481869, train_1259585693, train_2639768342, train_1358561112, train_1665938765, train_1519166571, train_998568945, train_1221993049, train_2434683709, train_1116319109, train_2379827418, train_1516573836, train_310896259, train_3933952478, train_1827504717, train_1005373935, train_3386243561, train_2067371060, train_3556479937, train_951314265, train_3453468105, train_2563204023]",[],"[train_2091196525, train_3386243561, train_3976679470, train_1911257755, train_3198005853, train_2406599165, train_1519166571, train_1516573836, train_379528076, train_2464356923, train_1930813487, train_586481869, train_59243023, train_4248767492, train_1806152124, train_2812635096, train_830884721, train_511692366, train_2288590299, train_798983928, train_1633843449, train_3369186413, train_951314265, train_3563363070, train_621862145, train_2639768342, train_998568945, train_1984944968, train_2658426624, train_4074846898, train_651460400]","[train_920661713, train_3453468105, train_2067371060, train_3210343892, train_1225784982, train_3383937433, train_1358561112, train_310896259, train_3933952478, train_1221993049, train_4061750790, train_2001760144, train_194519968, train_1512475953, train_2304760112, train_475857941, train_2434683709, train_3749487198, train_4200009730, train_3855055492, train_3261237801, train_1116319109, train_3412174951, train_1259585693, train_4030248161, train_4176745626, train_3346806089, train_3318627594, train_1835268339, train_2563204023, train_2379827418, train_400379362, train_3171305414, train_2330208712, train_1665938765, train_4095331020, train_3023369684, train_1005373935, train_3015101622, train_1292152697, train_4016582588, train_36508312, train_1827504717, train_1449806723, train_1038767026, train_2962857384, train_3556479937, train_3205349916, train_59612027]"
1,0.015152,"[train_4206743389, train_984412308, train_121852154, train_762535630, train_3517115705, train_2262735115, train_3755300225, train_368430234, t

In [50]:
train.sort_values("f1", ascending=True, inplace=True, ignore_index=True)
train[cols].head()

,f1,target,matches,phash_matches,title_matches,image_matches
0,0.015152,"[train_4206743389, train_984412308, train_121852154, train_762535630, train_3517115705, train_2262735115, train_3755300225, train_368430234, train_2317288464, train_3208392688, train_1458933053, train_4282331229, train_3949355563, train_1086352645, train_1446216827, train_2642455336, train_3838978688, train_98545496, train_3569591480, train_797513730, train_863149549, train_1580073987, train_2081303877, train_3194609880, train_678936454, train_2677777576, train_864736904, train_2184916710, train_1551941123, train_902429172, train_3338833167, train_1079287407, train_1966427691, train_2974564568, train_225670337, train_1905044128, train_1806160391, train_592604757, train_2128389901, train_4028145462, train_2333705544, train_686541160, train_2710828031, train_3340371184, train_1724556465, train_3003421228, train_1166656904, train_150004523, train_2057764463, train_1273401808, train_219281992]","[train_920661713, train_2001760144, train_4095331020, train_1292152697, train_1633843449, train_2812635096, train_1038767026, train_3318627594, train_2464356923, train_798983928, train_1930813487, train_4061750790, train_36508312, train_3023369684, train_621862145, train_3198005853, train_2091196525, train_59612027, train_3369186413, train_3563363070, train_1984944968, train_4074846898, train_1225784982, train_1806152124, train_2330208712, train_3976679470, train_3749487198, train_3346806089, train_3015101622, train_651460400, train_1449806723, train_4016582588, train_3210343892, train_3171305414, train_3261237801, train_400379362, train_2962857384, train_2406599165, train_3855055492, train_4030248161, train_2658426624, train_4248767492, train_1835268339, train_511692366, train_1911257755, train_3205349916, train_475857941, train_379528076, train_4200009730, train_2288590299, train_984412308, train_2304760112, train_59243023, train_194519968, train_3412174951, train_3383937433, train_1512475953, train_4176745626, train_830884721, train_586481869, train_1259585693, train_2639768342, train_1358561112, train_1665938765, train_1519166571, train_998568945, train_1221993049, train_2434683709, train_1116319109, train_2379827418, train_1516573836, train_310896259, train_3933952478, train_1827504717, train_1005373935, train_3386243561, train_2067371060, train_3556479937, train_951314265, train_3453468105, train_2563204023]",[],"[train_2091196525, train_3386243561, train_3976679470, train_1911257755, train_3198005853, train_2406599165, train_1519166571, train_1516573836, train_379528076, train_2464356923, train_1930813487, train_586481869, train_59243023, train_4248767492, train_1806152124, train_2812635096, train_830884721, train_511692366, train_2288590299, train_798983928, train_1633843449, train_3369186413, train_951314265, train_3563363070, train_621862145, train_2639768342, train_998568945, train_1984944968, train_2658426624, train_4074846898, train_651460400]","[train_920661713, train_3453468105, train_2067371060, train_3210343892, train_1225784982, train_3383937433, train_1358561112, train_310896259, train_3933952478, train_1221993049, train_4061750790, train_2001760144, train_194519968, train_1512475953, train_2304760112, train_475857941, train_2434683709, train_3749487198, train_4200009730, train_3855055492, train_3261237801, train_1116319109, train_3412174951, train_1259585693, train_4030248161, train_4176745626, train_3346806089, train_3318627594, train_1835268339, train_2563204023, train_2379827418, train_400379362, train_3171305414, train_2330208712, train_1665938765, train_4095331020, train_3023369684, train_1005373935, train_3015101622, train_1292152697, train_4016582588, train_36508312, train_1827504717, train_1449806723, train_1038767026, train_2962857384, train_3556479937, train_3205349916, train_59612027]"
1,0.015152,"[train_4206743389, train_984412308, train_121852154, train_762535630, train_3517115705, train_2262735115, train_3755300225, train_368430234, t